# Credit Card Fraud Detection

In [14]:
from pymongo import MongoClient
import pandas as pd
import os
from dotenv import load_dotenv

mongodb_uri = os.getenv("MONGODB_URI")

## Setup Connection to MongoDB database

In [16]:
connection_string = mongodb_uri
client = MongoClient(connection_string)
db = client.your_database_name

In [18]:
collection = db.your_collection_name
documents = collection.find()
df = pd.DataFrame(list(documents))
df.head(1)


""
